# Working with Visualizations and Maps
Using Holoviews and Folium and a small dataset describing Roman Amphitheaters. For more info see https://github.com/sfsheath/roman-amphitheaters .

This notebook creates three html files. See:

* http://sfsheath.github.io/amphitheaters-holoviews-folium/holoview-bar-by-modcountry.html
* http://sfsheath.github.io/amphitheaters-holoviews-folium/holoview-layout.html
* http://sfsheath.github.io/amphitheaters-holoviews-folium/folium-by-romanregion.html

Github doesn't display Holoviews or Folium output so I am posting this notebook with cells not run. Running the cells will display the html output inline. I probably could use nbviewer, but have chosen not to.

In [ ]:
import folium
import pandas as pd
import numpy as np
import holoviews as hv
hv.extension('bokeh')

from bokeh.models import HoverTool
from folium import FeatureGroup, LayerControl, Map, Marker

print(folium.__version__)
print(hv.__version__)
print(pd.__version__)


## Load Data

In [ ]:
ramphs_df = pd.read_csv('http://sebastianheath.com/roman-amphitheaters/roman-amphitheaters.csv')

## Holoviews

### Single Bar Chart of Seating Capacity by Country

In [ ]:
cap_hover = HoverTool(tooltips=[("capacity", "@capacity")])


In [ ]:
%%output filename="holoview-bar-by-modcountry" fig="html"
%%opts Bars [height=600 width=400 xrotation=90 tools=[cap_hover]]
# sum seating capacity by modern country .
summed = ramphs_df.groupby(by = 'modcountry', as_index=False).sum()\
   .sort_values(by = 'capacity', ascending = False)

r_bars = hv.Bars(summed[['modcountry','capacity']].query('capacity > 0'))

r_bars

## More 'Element' Types and Layouts

In [ ]:
r_table = hv.Table(ramphs_df[['label','modcountry']])

In [ ]:
r_hist = hv.Histogram(np.histogram(ramphs_df['capacity'].dropna(), bins=11), kdims=['capacity'])

In [ ]:
scatter_hover = HoverTool(tooltips=[("label", "@label"),
                            ("region","@romanregion"),
                            ("major axis","@extmajor"),
                            ("minor axis","@extminor")])

In [ ]:
%%opts Scatter [tools=[scatter_hover]] (size=5)

r_scatter = hv.Scatter(ramphs_df, kdims='extmajor', vdims=['extminor','label','romanregion'])

In [ ]:
%%opts BoxWhisker [xrotation=90]
r_boxwhisker = hv.BoxWhisker(ramphs_df.query('modcountry in ["France","Italy","Tunisia","United Kingdom"]'),
                             kdims = 'modcountry',
                             vdims = 'capacity')

#### Simple Table, Scatter plot of Exterior Dimenions, Histogram of Capacities, and BoxPlot from Four Countries

In [ ]:
%%output filename="holoview-layout" fig="html"

(r_table + r_scatter + r_hist + r_boxwhisker).cols(2)

## Interactive Map with Folium


In [ ]:
m = Map(
    location=[45.5,14],
    zoom_start=4,
    tiles='Stamen Watercolor'# what happens if you take this away?
)

template = "Label: <b>{}</b><br/>Region: <b>{}</b><br/>Capacity: <b>{}</b>"

# make FeatureGroups for each country
for c in ramphs_df.romanregion.sort_values().unique():
    q = "romanregion == '{}'".format(c)
    c_group = FeatureGroup(name = c)
    for index,row in ramphs_df.query(q).iterrows():
        x = row.longitude
        y = row.latitude

        capacity = 'n/a'
        if row.capacity > 0:
            capacity = int(row.capacity)
            
        folium.CircleMarker([y,x],radius= 2,
                            fill_color='red',
                            color = 'red',
                            popup=template.format(row.label, row.romanregion, capacity)).add_to(c_group)

        #print(p.label)
    c_group.add_to(m)
    
    
# add another tile layer
folium.TileLayer('Stamen Terrain').add_to(m)


# add layer control
LayerControl().add_to(m)

m.save('folium-by-romanregion.html')

m

# from IPython.display import IFrame
# IFrame('ramphs_by_region.html', width=650, height=450)